# 모델링

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/최애림/cakd7/m9_딥러닝/3차 프로젝트/5팀(윤정준, 최애림, 진청아, 이정현a)/코드

/content/drive/.shortcut-targets-by-id/1niQkF2oFYkuV2aahMfHbpa3kXs5yrcAd/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


In [ ]:
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions

base_model = tf.keras.applications.InceptionV3(input_shape=[32*3, 32*3, 3], include_top=False, weights='imagenet')
base_model.summary()

87910968/87910968 [==============================] - 3s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 47, 47, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 47, 47, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activatio

## 저장한 배열이미지 데이터 불러오기

In [ ]:
dataset_label = np.load('./array/1122_invasive_indicator.npy')
dataset_patches = np.load('./array/1122_real_invasive_indicator.npy')

time: 4.6 s (started: 2022-11-24 04:53:10 +00:00)


In [ ]:
dataset_patches.shape

(144362, 32, 32, 3)

time: 3.83 ms (started: 2022-11-24 04:53:32 +00:00)


### 침윤성 종양 여부에 따라 데이터 나누기

In [ ]:
patches_with_tumor = np.array([dataset_patches[ind, :, :] for ind in range(len(dataset_label)) if dataset_label[ind] == 1])
patches_without_tumor = np.array([dataset_patches[ind, :, :] for ind in range(len(dataset_label)) if dataset_label[ind] == 0])

In [ ]:
# 침윤성 종양 부분
plt.figure(figsize = (20,8))
for i in range(40):
  ax = plt.subplot(4, 10, i + 1)
  plt.imshow(patches_with_tumor[i])

In [ ]:
# 비침윤성 종양 부분
plt.figure(figsize = (20,8))
for i in range(40):
  ax = plt.subplot(4, 10, i + 1)
  plt.imshow(patches_without_tumor[i])

### 훈련 및 테스트용 데이터 분리

In [ ]:
# 훈련 테스트용 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(dataset_patches, 
                                                  dataset_label, 
                                                  test_size=0.2, 
                                                  random_state=42)
X_train = (np.array(X_train).astype('float32') / 255.) * 2.0 - 1
X_val = (np.array(X_val).astype('float32') / 255.) * 2.0 - 1.0

# 확인
print("Size of the train set: {:d}".format(len(y_train)))
print("Size of the validation set: {:d}".format(len(y_val)))
print("Size of the train data sample: " + str(X_train[0].shape))
print("Size of the validation data sample: " + str(X_val[0].shape))

Size of the train set: 115489
Size of the validation set: 28873
Size of the train data sample: (32, 32, 3)
Size of the validation data sample: (32, 32, 3)
time: 1.26 s (started: 2022-11-23 09:02:15 +00:00)


### 모델링

In [ ]:
import keras

# 기존에 학습된 모델 그대로 활용
base_model.trainable = False 

# input layer 추가
input_layer = tf.keras.layers.InputLayer(
                                          input_shape=[32, 32, 3])
                                          input_tranConv = tf.keras.layers.Conv2DTranspose(3, [3, 3], strides=3,
                                                                          padding='same',
                                                                          use_bias=False)

# output layer 추가하기
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')


# 최종 모델 layer 구성
model = tf.keras.Sequential([
                              input_layer,
                              input_tranConv,                           
                              base_model,
                              global_average_layer,
                              prediction_layer
])

# 모델 컴파일
early_stopping_cb = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True, monitor='val_loss')
optimizer = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(optimizer = optimizer,
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

time: 1.77 s (started: 2022-11-23 09:04:53 +00:00)


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose_2 (Conv2DT  (None, 96, 96, 3)        81        
 ranspose)                                                       
                                                                 
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 21,804,914
Trainable params: 2,130
Non-trainable params: 21,802,784
_________________________________________________________________
time: 59.8 ms (started: 2022-11-23 09:

### 학습

In [ ]:
EPOCH = 10
BATCH_SIZE = 512
y_train = np.array(y_train)
y_val = np.array(y_val)

history = model.fit(X_train, y_train, 
                    validation_data = (X_val, y_val), 
                    epochs = EPOCH, 
                    callbacks = [early_stopping_cb], 
                    batch_size=BATCH_SIZE)

Epoch 1/10
226/226 [==============================] - 124s 457ms/step - loss: 0.1498 - accuracy: 0.9665 - val_loss: 0.1488 - val_accuracy: 0.9655
Epoch 2/10
226/226 [==============================] - 96s 425ms/step - loss: 0.1429 - accuracy: 0.9669 - val_loss: 0.1509 - val_accuracy: 0.9655
Epoch 3/10
226/226 [==============================] - 99s 438ms/step - loss: 0.1422 - accuracy: 0.9669 - val_loss: 0.1479 - val_accuracy: 0.9655
Epoch 4/10
226/226 [==============================] - 96s 424ms/step - loss: 0.1415 - accuracy: 0.9669 - val_loss: 0.1490 - val_accuracy: 0.9655
Epoch 5/10
226/226 [==============================] - 96s 424ms/step - loss: 0.1406 - accuracy: 0.9669 - val_loss: 0.1482 - val_accuracy: 0.9655
Epoch 6/10
226/226 [==============================] - 99s 437ms/step - loss: 0.1411 - accuracy: 0.9669 - val_loss: 0.1467 - val_accuracy: 0.9655
Epoch 7/10
226/226 [==============================] - 96s 424ms/step - loss: 0.1402 - accuracy: 0.9669 - val_loss: 0.1480 - val_a

In [ ]:
model.save('./models/1122_patches_test_model.h5')

time: 764 ms (started: 2022-11-22 08:03:00 +00:00)


In [ ]:
model.save_weights('./models/1122_patches_test_model_weights.h5')

time: 716 ms (started: 2022-11-22 08:03:00 +00:00)


In [ ]:
# 재학습이 불가하도록 한 layer의 index 반환
layer_freeze = "mixed0"

for index, layer in enumerate(base_model.layers):
    if layer.name == layer_freeze:
        print(str(index) + ":" + str(layer.name))

40:mixed0
time: 1.41 ms (started: 2022-11-23 09:23:05 +00:00)


In [ ]:
# index 이후 layer 부터 재학습 가능하도록 함
model.trainable = True
max_layer_freeze = 40

for layer in base_model.layers[:max_layer_freeze]:
  layer.trainable = False

time: 13.7 ms (started: 2022-11-23 09:23:05 +00:00)


In [ ]:
# checkpoint 디렉토리 생성
drive_root = './'
checkpoint_dir = os.path.join(drive_root, "Checkpoints", "InceptionV3-Freeze40-UnbalData")

# 폴더 생성
print("Checkpoints directory is", checkpoint_dir)
if os.path.exists(checkpoint_dir):
  print("Checkpoints folder already exists")
else:
  print("Creating a checkpoints directory")
  os.makedirs(checkpoint_dir)

# checkpoint callback 생성
checkpoint_path = os.path.join(checkpoint_dir, "best_model.hdf5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                    filepath=checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True,
                    save_weights_only=False, mode='auto')

Checkpoints directory is ./Checkpoints/InceptionV3-Freeze40-UnbalData
Checkpoints folder already exists
time: 2.25 ms (started: 2022-11-23 09:23:05 +00:00)


In [ ]:
# 데이터 증식
from keras.preprocessing.image import ImageDataGenerator

# 이미지증식 제너레이터 생성
train_datagen = ImageDataGenerator(
                                    shear_range=0.2,    
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,  
                                    horizontal_flip=True,
                                    vertical_flip=True)

train_datagen.fit(X_train)

# 증식(validation 데이터는 증식하지 않도록 주의)
train_generator = train_datagen.flow(
                                      X_train,
                                      y_train,
                                      batch_size=BATCH_SIZE)

time: 1.51 s (started: 2022-11-23 09:23:05 +00:00)


In [ ]:
# 모델 컴파일

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


# 학습
EPOCH = 200
history = model.fit(
      train_generator,
      steps_per_epoch=X_train.shape[0] / BATCH_SIZE,
      epochs = EPOCH,
      validation_data=(X_val, y_val), 
      callbacks=[checkpoint_callback, early_stopping_cb])   

Epoch 1/200
225/225 [==============================] - 146s 609ms/step - loss: 0.1541 - accuracy: 0.9663 - val_loss: 0.1463 - val_accuracy: 0.9655
Epoch 2/200
225/225 [==============================] - 129s 572ms/step - loss: 0.1425 - accuracy: 0.9669 - val_loss: 0.1461 - val_accuracy: 0.9655
Epoch 3/200
225/225 [==============================] - 129s 571ms/step - loss: 0.1405 - accuracy: 0.9669 - val_loss: 0.1446 - val_accuracy: 0.9655
Epoch 4/200
225/225 [==============================] - 131s 580ms/step - loss: 0.1398 - accuracy: 0.9669 - val_loss: 0.1472 - val_accuracy: 0.9655
Epoch 5/200
225/225 [==============================] - 129s 571ms/step - loss: 0.1395 - accuracy: 0.9669 - val_loss: 0.1437 - val_accuracy: 0.9655
Epoch 6/200
225/225 [==============================] - 131s 579ms/step - loss: 0.1384 - accuracy: 0.9669 - val_loss: 0.1443 - val_accuracy: 0.9655
Epoch 7/200
225/225 [==============================] - 131s 579ms/step - loss: 0.1382 - accuracy: 0.9669 - val_loss: 0

### 저장하기

In [ ]:
model.save('./models/1122_image_inceptionv3_method3.h5')

### 모델 불러오기

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('./models/1121_image_inceptionv3_method3.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTra  (None, 96, 96, 3)        81        
 nspose)                                                         
                                                                 
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 21,804,914
Trainable params: 21,342,770
Non-trainable params: 462,144
_________________________________________________________________
time: 4.52 s (started: 2022-11-22 06:3

## 예측 TEST

### 패치데이터 불러오기

In [ ]:
train_patches = np.load('./array/1121_real_invasive_indicator_train_2개.npy', allow_pickle= True)
train_patches

In [ ]:
train_patches[0][0].shape

(32, 32, 3)

time: 3.77 ms (started: 2022-11-22 06:37:37 +00:00)


In [ ]:
for i in range(2):
  print(f'***patch{i+1}번째')
  invasive = 0
  non_invasive = 0
  for patch in train_patches[i]:
    patch = (patch.astype('float32') / 255.) * 2.0 - 1
    preds = model.predict(patch[None,:], verbose = 0)
    
    if preds > 0.5:
      invasive += 1
    else:
      non_invasive += 1
    
  print(f'invasive 개수: {invasive}')
  print(f'non_invasive 개수: {non_invasive}')

***patch1번째
invasive 개수: 3981
non_invasive 개수: 90
***patch2번째
invasive 개수: 2440
non_invasive 개수: 9
time: 6min 52s (started: 2022-11-22 06:51:32 +00:00)
